In [9]:
import geopandas as gpd
import pandas as pd
from geetiles import cmds
import numpy as np

In [2]:
mfile = '/opt/data/california-worldcover-chips-osm-multilabels.parquet'

In [13]:
data = gpd.read_parquet(mfile)

In [14]:
data.head()

,onehot_label,string_labels,col,row,geometry
chip_id,,,,,
2d020602331e4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[highway, highway=track]",122624,111104,"POLYGON ((-114.76000 32.72000, -114.76000 32.7..."
34c5edf7ed602,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[amenity, amenity=parking, highway, highway=tr...",122624,110848,"POLYGON ((-114.76000 32.74133, -114.76000 32.7..."
370214ce19e1d,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[highway, highway=track, highway=motorway, hig...",122368,110848,"POLYGON ((-114.78133 32.74133, -114.78133 32.7..."
371eeed5e3259,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[highway, highway=track, highway=motorway, hig...",122112,110848,"POLYGON ((-114.80267 32.74133, -114.80267 32.7..."
24763185559b0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[highway, highway=track]",122112,110592,"POLYGON ((-114.80267 32.76267, -114.80267 32.7..."


In [15]:
# prepare for geetiles

tmpdata = data[['geometry']].copy()
tmpdata['identifier'] = tmpdata.index
tmpdata.to_file("/tmp/data.geojson", driver='GeoJSON')

## call geetiles to make the split

In [33]:
cmds.split(tiles_file = '/tmp/data.geojson',
                nbands = 80,
                angle = -np.pi/4,
                train_pct = 0.7,
                test_pct = 0.15,
                val_pct = 0.15)

saved to /tmp/data.geojson
all splits saved to /tmp/data_splits.csv


In [34]:
splits = pd.read_csv("/tmp/data_splits.csv").set_index('identifier')
splits.index.name = 'chip_id'
splits.shape

(83609, 1)

In [35]:
data.head()

,onehot_label,string_labels,col,row,geometry
chip_id,,,,,
2d020602331e4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[highway, highway=track]",122624,111104,"POLYGON ((-114.76000 32.72000, -114.76000 32.7..."
34c5edf7ed602,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[amenity, amenity=parking, highway, highway=tr...",122624,110848,"POLYGON ((-114.76000 32.74133, -114.76000 32.7..."
370214ce19e1d,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[highway, highway=track, highway=motorway, hig...",122368,110848,"POLYGON ((-114.78133 32.74133, -114.78133 32.7..."
371eeed5e3259,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[highway, highway=track, highway=motorway, hig...",122112,110848,"POLYGON ((-114.80267 32.74133, -114.80267 32.7..."
24763185559b0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[highway, highway=track]",122112,110592,"POLYGON ((-114.80267 32.76267, -114.80267 32.7..."


In [36]:
data.join(splits).to_parquet(mfile)